## 元論文

In [61]:
import numpy as np
import pandas as pd
from collections import defaultdict
import sys
import os

sys.path.append(os.path.abspath(".."))  # 1階層上を追加
from pulp_exc.scflp_v2 import SCFLPData, SCFLPSolver, make_random_instance

In [62]:
"""instances = [
    {"I": 200,  "J": 200,  "p": 2, "r": 2},
    {"I": 300,  "J": 300,  "p": 2, "r": 2},
    {"I": 400,  "J": 400,  "p": 2, "r": 2},
    {"I": 500,  "J": 500,  "p": 2, "r": 2},
    {"I": 600,  "J": 600,  "p": 2, "r": 2},
    {"I": 700,  "J": 700,  "p": 2, "r": 2},
    {"I": 800,  "J": 800,  "p": 2, "r": 2},
    {"I": 900,  "J": 900,  "p": 2, "r": 2},
    {"I": 1000,  "J": 1000,  "p": 2, "r": 2},
]
"""

instances = [
    {"I": 20,  "J": 20,  "p": 2, "r": 2},
    {"I": 20,  "J": 20,  "p": 3, "r": 2},
    {"I": 20,  "J": 20,  "p": 2, "r": 3},

    {"I": 40,  "J": 40,  "p": 2, "r": 2},
    {"I": 40,  "J": 40,  "p": 3, "r": 2},
    {"I": 40,  "J": 40,  "p": 2, "r": 3},

    {"I": 60,  "J": 60,  "p": 2, "r": 2},
    {"I": 60,  "J": 60,  "p": 3, "r": 2},
    {"I": 60,  "J": 60,  "p": 2, "r": 3},

    {"I": 80,  "J": 80,  "p": 2, "r": 2},
    {"I": 80,  "J": 80,  "p": 3, "r": 2},
    {"I": 80,  "J": 80,  "p": 2, "r": 3},

    {"I": 100, "J": 100, "p": 2, "r": 2},
    {"I": 100, "J": 100, "p": 3, "r": 2},
    {"I": 100, "J": 100, "p": 2, "r": 3},
]

"""instances_2 = [
    {"I": 200,  "J": 200,  "p": 2, "r": 2},
    {"I": 300,  "J": 300,  "p": 2, "r": 2},
    {"I": 400,  "J": 400,  "p": 2, "r": 2},
    {"I": 500,  "J": 500,  "p": 2, "r": 2},
]"""


'instances_2 = [\n    {"I": 200,  "J": 200,  "p": 2, "r": 2},\n    {"I": 300,  "J": 300,  "p": 2, "r": 2},\n    {"I": 400,  "J": 400,  "p": 2, "r": 2},\n    {"I": 500,  "J": 500,  "p": 2, "r": 2},\n]'

In [63]:
n_runs = 10
beta = 0.1
all_results = []

for inst_id, params in enumerate(instances, start=1):
    for run in range(n_runs):
        seed = np.random.SeedSequence()
        data = make_random_instance(**params, beta=beta, seed=seed)
        
        solver = SCFLPSolver(data, milp_backend="pulp", pulp_solver="CBC")
        result = solver.solve(max_rounds=300, tol=1e-9, verbose=False)

        # 各試行ごとに記録
        row = {
            "instance_id": inst_id,
            "run": run,
            "I": params["I"],
            "J": params["J"],
            "p": params["p"],
            "r": params["r"],
            "beta": beta,
            "theta": result["theta"],
            "time_sec": result["time_sec"],
            "status": result.get("status", None),
            "iterations": result.get("iterations", None),
            "cuts": result.get("cuts", None),
            "obj": result.get("obj", None),
        }
        all_results.append(row)

# DataFrame にまとめて CSV 保存
df = pd.DataFrame(all_results)
df.to_csv("scflp_results_all_over100_with_previous_to1000.csv", index=False, encoding="utf-8")

print("保存完了: scflp_results_all_over100_with_previous_to1000.csv")

════════════════════════════════════════════════════════════════════════════════
🚀 S-CFLP を解きます（DCG + Bulge/Submodular カット）
════════════════════════════════════════════════════════════════════════════════
➤ 候補地 J = 20, 需要点 I = 20, p = 2, r = 2
────────────────────────────────────────────────────────────────────────────────
➤ 🧮 ラウンド 1: master 解 → \hat(θ) = 1.000000, |x| = 2/2（整数）
➤ 🔎 分離：ŷ は [1, 15]（r = 2）。上界 α − βᵀŷ = 0.685388
➤ 📐 評価：L(x̂,ŷ) = 0.453073 → 違反量 \hat(θ) - L = 5.469e-01
✂️  カット追加: BULGE + SUBMOD
────────────────────────────────────────────────────────────────────────────────
➤ 🧮 ラウンド 2: master 解 → \hat(θ) = 1.000000, |x| = 2/2（整数）
➤ 🔎 分離：ŷ は [0, 12]（r = 2）。上界 α − βᵀŷ = 0.694543
➤ 📐 評価：L(x̂,ŷ) = 0.494723 → 違反量 \hat(θ) - L = 5.053e-01
✂️  カット追加: BULGE + SUBMOD
────────────────────────────────────────────────────────────────────────────────
➤ 🧮 ラウンド 3: master 解 → \hat(θ) = 0.776102, |x| = 2/2（整数）
➤ 🔎 分離：ŷ は [0, 15]（r = 2）。上界 α − βᵀŷ = 0.673327
➤ 📐 評価：L(x̂,ŷ) = 0.541837

KeyboardInterrupt: 

## 豊島

In [ ]:
%run ..\mutaion_gda\func_v4.ipynb

In [ ]:
import numpy as np
import pandas as pd
import random, time

results = []
num_rows_columns = 50   # サイト配置グリッドの一辺
alpha = 0.0             # 固定
beta = 0.1
n_runs = 10             # 繰り返し回数

for inst_id, params in enumerate(instances, start=1):
    objs_relaxed, objs_binary, objs_ex, times = [], [], [], []

    for run in range(n_runs):
        seed_int = random.getrandbits(32)

        # 需要点の重みベクトル
        h_i = np.full(params["I"], 1 / params["I"])
        J_L, J_F = set(), set()

        start = time.time()
        x_R, y_R, obj_relaxed, x_proj, y_proj, obj_binary, obj_ex, candidate_sites, demand_points, history = lgda_solver(
            params["I"], params["J"], num_rows_columns,
            params["p"], params["r"],
            alpha, beta, h_i, J_L, J_F,
            eta_x=0.01, eta_y=0.01,
            mu=.5,
            max_iter=100_000,
            tau_interval=2000,
            return_history=True
        )
        elapsed = time.time() - start

        objs_relaxed.append(obj_relaxed)
        objs_binary.append(obj_binary)
        objs_ex.append(obj_ex)
        times.append(elapsed)

    # 各インスタンスの平均を保存
    row = {
        "I": params["I"],
        "J": params["J"],
        "p": params["p"],
        "r": params["r"],
        #"beta": beta,
        "obj_relaxed_avg": np.mean(objs_relaxed),
        "obj_binary_avg": np.mean(objs_binary),
        "obj_ex_avg": np.mean(objs_ex),
        "time_avg_sec": np.mean(times)
    }
    results.append(row)

# DataFrame 化 & CSV 保存
df = pd.DataFrame(results)
df.to_csv("lgda_results_avg_over100_with_previous_to1000.csv", index=False, encoding="utf-8")
print(df)
print("保存完了: lgda_results_avg_over100_with_previous_to1000.csv")

1988821601
1061834338
2637971022
3502224108
1167352982
2822576132
3366382298
1460538567
1669138714
3803015618
2585865977
289134523
2983046046
3459110974
2440168459
168156685
3571006373
3917165069
798558292
103835197
26058117
3607075197
4237543752
1366784642
1778934372
2324549970
1763895840
434656439
464539537
3609501913
3350060469
2948938943
2845522705
784306112
2329975622
1106344333
4184322228
2270939136
2610001920
2692397804
1204613087
834038367
3711606688
4034786397
2803140736
3689150596
1113485577
910474560
1115380408
2860915839
2490268259
1056106088
1168711361
292663004
3761957060
1353871361
3395465585
1180247788
3702120596
3412361815
226214457
3220773480
1434031805
204729833
3282158789
2278221992
3676915680
551353890
2447458197
3723267176
2879669324
899199989
3235009434
2294637551
1639528053
1631795143
3087068271


KeyboardInterrupt: 

In [ ]:
import pandas as pd
from pathlib import Path

# 入出力パスを指定
in_path = Path("result.csv")              # ← 元のCSVパスに置き換え
out_path = Path("results_rounded.csv")     # ← 出力CSV名

# 四捨五入する列
target_cols = ["obj(MIP)", "obj(Mutation)"]
sec = ["time(MIP)", "time(Mutation)"]
# 読み込み
df = pd.read_csv(in_path)

# 必要列の存在チェック
missing = [c for c in target_cols if c not in df.columns]
if missing:
    raise ValueError(f"次の列が見つかりません: {missing}")

# 数値化して小数第4位で四捨五入
for c in target_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce").round(3)
    
# 数値化して小数第4位で四捨五入
for c in sec:
    df[c] = pd.to_numeric(df[c], errors="coerce").astype(int)

# 保存（インデックス列は付けない）
df.to_csv(out_path, index=False)

print(f"保存しました: {out_path.resolve()}")


保存しました: C:\Users\theko\allocate-probrem\experiment\results_rounded.csv
